In [16]:
import warnings
# Suppress specific deprecation warnings from botocore
warnings.filterwarnings("ignore", message="datetime.datetime.utcnow()", category=DeprecationWarning)

# Feature Store apply

In [14]:
# registry.db 확인
!ls feature_repo/registry.db

ls: feature_repo/registry.db: No such file or directory


In [17]:
# init registry and online store
!cd feature_repo && feast apply

No project found in the repository. Using project name credit_scoring_aws defined in feature_store.yaml
Applying changes for project credit_scoring_aws
/Users/choisungwook/git/portfolio/mlops/feast-aws-credit-scoring-tutorial/.venv/lib/python3.12/site-packages/botocore/auth.py:425: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  datetime_now = datetime.datetime.utcnow()
Deploying infrastructure for zipcode_features
Deploying infrastructure for credit_history


In [20]:
# registry.db 확인
!ls feature_repo/registry.db

feature_repo/registry.db


In [19]:
# Online Feature Store 확인
!aws dynamodb list-tables --region ap-northeast-2

{
    "TableNames": [
        "credit_scoring_aws.credit_history",
        "credit_scoring_aws.zipcode_features"
    ]
}


# Feature Store Registry 확인

In [5]:
!cd feature_repo && \
  echo "[info] feast entities list" && \
  feast entities list && \
  echo "\n" && \
  echo "[info] feast feature-views list" && \
  feast feature-views list

[info] feast entities list
NAME     DESCRIPTION                                                   TYPE
zipcode                                                                ValueType.INT64
dob_ssn  Date of birth and last four digits of social security number  ValueType.STRING


[info] feast feature-views list
NAME              ENTITIES     TYPE
zipcode_features  {'zipcode'}  FeatureView
credit_history    {'dob_ssn'}  FeatureView


# Training

In [21]:
import pandas as pd
from credit_model import CreditScoringModel

# Get historic loan data
loans = pd.read_parquet("data/loan_table.parquet")

# Create model
model = CreditScoringModel()

# Train model (using Redshift for zipcode and credit history features)
if not model.is_model_trained():
    print("[info] Training model with historic loan data...")
    print("[info] Train successful.")
    model.train(loans)
else:
    print("[info] model file is already exist. Skipping training...")

print("[info] Done")

[info] model file is already exist. Skipping training...
[info] Done


# Predict

In [22]:
import sys
from credit_model import CreditScoringModel


model = CreditScoringModel()
if not model.is_model_trained():
    sys.exit("Model is not trained. Please run the training script first.")

# Make online prediction (using DynamoDB for retrieving online features)
loan_request = {
    "zipcode": [76104],
    "dob_ssn": ["19630621_4278"],
    "person_age": [133],
    "person_income": [59000],
    "person_home_ownership": ["RENT"],
    "person_emp_length": [123.0],
    "loan_intent": ["PERSONAL"],
    "loan_amnt": [35000],
    "loan_int_rate": [16.02],
}

result = model.predict(loan_request)

if result == 0:
    print("Loan approved!")
elif result == 1:
    print("Loan rejected!")


Loan rejected!
